In [483]:
from lxml import html, etree
from io import StringIO
import re

# from .WordMonthExtractor import WordMonthExtractor
# from .HalfYearExtractor import HalfYearExtractor
# from .FromToExtractor import FromToExtractor
# from .ReleaseDateExtractor import get_release_date

class HocrParser:

    def __init__(self):
        self.doc = None
        self.root = None
        self.parsed_document = None


    def read_file(self, path):
        with open(path, 'r', encoding='utf-8', errors='ignore') as file:
            lines = file.readlines()
            first_line_i = 0
            for i, line in enumerate(lines):
                if line.find("xml version") > -1:
                    first_line_i = i
                    break
            xml_string = "".join(lines[first_line_i + 1:])
            parser = etree.HTMLParser()
            self.doc = etree.parse(StringIO(xml_string), parser)
            self.root = self.doc.getroot()
            del lines


    def parse_(self):
        document = []
        pages = [page for page in next(x for x in self.root if x.tag == 'body') if page.attrib.get('class', "") == 'ocr_page']
        for page in pages:
            page_l = []
            lines = [line for line in page if line.attrib.get('class', "") == 'ocrx_line']
            for _, line in enumerate(lines):
                words = [word.text for word in line if
                         word.attrib.get('class', "") == 'ocrx_word' and type(word.text) == str]
                if words:
                    line_joined = " ".join(words)
                page_l.append(line_joined)
            document.append(page_l)
        self.parsed_document = document
        return document

#     def get_company(self):
#         print("dupa")

    def get_company(self):
        company=''
        flag=0
        r = re.compile('spół*|kapita*|finanso*')
        forma=['s.a.','spółka akcyjna']
        text=[j for i in self.parsed_document for j in i]
        for x in text:
            #print(x.lower())
            for i in x.lower().split():
                 if i in forma and flag==0:
                        company=(x.split()[:x.lower().split().index(i)])
                        flag=1
        company=" ".join(company)
        syf_idx = [i for i, item in enumerate(company.lower().split()) if re.search(r'spół*|kapita*|finanso*|the|(\w+:)|^of$|działalności|pół*|zgromadzenia|(\w+czny)|zarządu|^do$|^o$|grupa|roku|dominują*', item)]
        if len(syf_idx)>0:
            return " ".join(company.split()[syf_idx[len(syf_idx)-1]+1:])
        else:
            return " ".join(company.split())
        
    def get_release_date(self):
        return get_release_date(self.parsed_document)

#     def get_dates(self):
#         text = " ".join(self.parsed_document[0][0:15])
#         text = text.lower()
        
#         extractors = [
#             HalfYearExtractor(), 
#             WordMonthExtractor(), 
#             FromToExtractor()
#             ]

#         for extractor in extractors:
#             dates = extractor.extract(text)
#             if dates is not None:
#                 return dates[0].isoformat(), dates[1].isoformat()
        
#         return None

####### Example usage: #######
# parser = HocrParser()
# parser.read_file("./data/contest/train/reports/208910/ZMR_PSr_2012_SPRAWOZDANIE_ZARZADU.hocr")
# parser.read_file("./data/contest/train/reports/15988/rozszerzone_skonsolidowane_sprawozdanie__finansowe_Grupy_Kapitalowej_ProchemSA_na_30.06.2005.hocr")
# document = parser.parse_()
# release_date = parser.get_release_date()
# dates = parser.get_dates()
#company_names=parser.get_company()

In [449]:
def get_company(document):
    company=''
    flag=0
    r = re.compile('spół*|kapita*|finanso*')
    forma=['s.a.','spółka akcyjna']
    text=[j for i in document for j in i]
    for x in text:
        for i in x.lower().split():
             if i in forma and flag==0:
                    company=(x.split()[:x.lower().split().index(i)])
                    flag=1
    company=" ".join(company)
    syf_idx = [i for i, item in enumerate(company.lower().split()) if re.search(r'spół*|kapita*|finanso*|the|(\w+:)|^of$|działalności|pół*|zgromadzenia|(\w+czny)|zarządu|^do$|^o$|grupa|roku|dominują*', item)]
    if len(syf_idx)>0:
        return " ".join(company.split()[syf_idx[len(syf_idx)-1]+1:])
    else:
        return " ".join(company.split())
    #print(company.split()[3])
    #zmianami, Zarząd Spółki Dominującej - Chemoservis-Dwory

In [484]:
import os
import csv
company_names=[]
dir_names=[]
rootdir = r'C:\PW\NLP\contest\train'
for subdir, dirs, files in os.walk(rootdir):
    #filename = os.fsdecode(files)
    for file in files:
        filename = os.fsdecode(file)
        if filename.endswith(".hocr"):
            filepath=os.path.join(subdir, file)
            dir_names.append(os.path.basename(subdir))
            #print(filepath)
            parser = HocrParser()
            parser.read_file(filepath)
            doc=parser.parse_()
            #get_company(doc)
            company_names.append(parser.get_company())
            #company_names.append(get_company(doc))
            #print(company_names)


In [485]:
import pandas as pd
import numpy as np

df=pd.DataFrame(list(zip(dir_names, company_names)))
#df.to_csv('output.csv', sep=';', encoding='utf-8',index=False)


truth=pd.read_csv('ground_truth-train.csv', sep=';')
truth=truth.iloc[:,0:2]
df.columns = ['id', 'company_pred']

df["id"] = pd.to_numeric(df["id"])
df["company_pred"] = df["company_pred"].astype('str') 
#truth.set_index('id').join(df.set_index('id'))
merged=df.join(truth.set_index('id'), on='id')
merged[['company']]
#merged['company']= merged['company'].map(lambda x: x.rstrip(' SA$ | S.A.$' | S.A$))
merged['company'] = merged['company'].astype(str)
merged['company_pred'] = merged['company_pred'].astype(str)
merged['company'].replace(regex=True,inplace=True,to_replace=r' SA| S.A.| Spółka Akcyjna| SPÓŁKA AKCYJNA| SPÓLKA AKCYJNA',value=r'')#.str.upper() #|Spółka Akcyjna|SPÓŁKA AKCYJNA$
merged['company_pred'] = merged['company_pred'].str.upper()
merged['company'] = merged['company'].str.upper()
merged['company'] = merged['company'].replace(' - ', '-')
merged['company'] = merged['company'].replace('"', '')
merged['company_pred'] = merged['company_pred'].replace('"', '')

#df.iloc[:, i] = df.iloc[:, i].str.replace('"', '')
merged.to_csv('compare.csv', sep=';', encoding='utf-8',index=False)

In [451]:
parser = HocrParser()
#parser.read_file("./data/contest/train/reports/208910/ZMR_PSr_2012_SPRAWOZDANIE_ZARZADU.hocr")
parser.read_file(r'C:\PW\NLP\contest\train\reports\367072\Abadon_Real_Estate_S.A._skonsolidowany_raport_-_I_polrocze_2017.hocr')

document = parser.parse_()
#release_date = parser.get_release_date()
#423396;WYBRANE DANE FINANSOWE INTROL;INTROL

In [452]:
get_company(document)

'ABADON REAL ESTATE'

In [380]:
# śmieciiiii


#document[0][3].split()[2]=='S.A.'
#document[1][1].split()
document[0][3]

#abc=re.search("S.A.", document[0][3])
#print(abc)

forma=['s.a.','spółka akcyjna','(„emitent”)']
text=[j for i in document for j in i]

company=''
flag=0
for x in text:
    #print(x.lower())
    for i in x.lower().split():
         if i in forma and flag==0:
                company=(x.split()[:x.lower().split().index(i)])
                flag=1
company=" ".join(company)
r = re.compile('spół*|kapita*|finanso*')
#newlist = list(filter(r.match, company.lower().split()))

syf_idx = [i for i, item in enumerate(company.lower().split()) if re.search(r'spół*|kapita*|finanso*|\b(\w+czny)\b', item)]

print(syf_idx)
if len(syf_idx)>0:
    print(" ".join(company.split()[syf_idx[len(syf_idx)-1]+1:]))
else:
    " ".join(company.split())
#Jednostką Dominującą w Grupie Kapitałowej Work Service jest spółka Work Service

[1]
KREDYT INKASO
